In [10]:
import numpy as np
import IPython
import time
from scipy.io import wavfile
from scipy.signal import stft
from scipy.signal import istft

In [11]:
# Esta función de no debe ser modificada.
def WAV_to_Array(WAV_file):
    sample_rate, data = wavfile.read(WAV_file)
    if(len(data.shape) > 1): # En caso de que tenga más de un canal de audio, se trabaja con el primero
        data = data[:,0]
    
    return sample_rate, data

# Esta función de no debe ser modificada.
def play_audio(data, sample_rate):
    IPython.display.display(IPython.display.Audio(data, rate=sample_rate))
    return

# Esta función de no debe ser modificada.
def STFT(data):
    _, _, M = stft(data)
    return M
    
# Esta función de no debe ser modificada.
def ISTFT(M):
    _, data = istft(M)
    return data

In [12]:
sample_rate, data = WAV_to_Array("original.wav")
d = STFT(data=data)

## Tests

In [13]:
def Split(M):

    M_Re = list()
    M_Im = list()

    for i in range(len(M)):
        real = list()
        img = list()
        for j in range(len(M[i])):
            real.append(np.real(M[i][j]))
            img.append(np.imag(M[i][j]))
        M_Re.append(real)
        M_Im.append(img)
    
    M_Re = np.array(M_Re)
    M_Im = np.array(M_Im)

    return M_Re, M_Im

In [14]:
def Merge(M_Re, M_Im):
    M = M_Re + M_Im*1j

    return M

In [15]:
# def Split(M):
#     M_Re = np.real(M).astype(np.float64)
#     M_Im = np.imag(M).astype(np.float64)
#     return M_Re, M_Im

In [16]:
M_Re, M_Im = Split(d)
s = Merge(M_Re, M_Im)

In [17]:
comparison = d == s
equal_arrays = comparison.all()
print(equal_arrays)

True


In [31]:
def PCA_SVD(M, m):
    # Apply PCA
    mu = np.mean(M, axis=0)
    Z = M - mu

    u, s, vh = np.linalg.svd(Z, full_matrices=False)
    
    V = vh.T[:, :m]
    Y = Z.dot(V)

    return  V, Y, mu

In [32]:
V, Y, mu = PCA_SVD(M_Re, 100)
print(V)

[[ 4.4385198e-17  5.9151150e-17 -1.7692980e-17 ...  1.0043935e-14
   3.5425930e-14  3.1209298e-14]
 [ 2.8311219e-16  1.4796842e-16  3.2039372e-17 ... -6.1240597e-14
  -8.7941583e-14 -1.0363240e-13]
 [ 7.1310585e-17 -1.0629442e-17  2.5913291e-17 ...  1.8930137e-13
   2.6249472e-13  3.2431843e-13]
 ...
 [ 5.2774008e-03  1.6917860e-03 -2.8191059e-04 ... -4.5641870e-03
   1.0307319e-03 -9.2870109e-03]
 [ 2.9871315e-03  7.7757137e-03  5.6850561e-03 ...  1.2897413e-03
   3.2249775e-02  5.0896765e-03]
 [ 7.7815233e-03 -7.6680477e-03  7.5204768e-03 ... -6.0714269e-03
   4.1751387e-03 -3.6453521e-03]]


In [20]:
def PCA_M(V, Y, mu):
    Mm = Y.dot(V.T) + mu
    return Mm

In [21]:
Mn = PCA_M(V, Y, mu)
Mn

array([[-1.2417633e-13,  3.9074560e-13, -1.0657686e-12, ...,
         1.0665648e+03,  7.4815857e+02,  1.4664443e+03],
       [ 1.7648544e-11,  5.8400333e-13, -1.6568977e-12, ...,
         4.0342776e+02,  7.8707562e+02, -5.7854077e+02],
       [-1.2035742e-11, -9.0783235e-11, -1.6569732e-12, ...,
        -1.8170269e+03, -1.8605286e+03, -1.0886995e+03],
       ...,
       [-1.4686894e-14,  9.2371315e-15, -2.7194501e-14, ...,
        -1.5792847e-03, -3.8022995e-03,  2.3394390e+01],
       [ 1.7944922e-14, -1.8974270e-14,  5.5961149e-14, ...,
         2.0427704e-03,  8.0320835e-03, -2.3398735e+01],
       [-1.9119519e-14,  2.3431135e-14, -7.0490909e-14, ...,
        -1.8506050e-03, -1.1823177e-02,  2.3401966e+01]], dtype=float32)

In [22]:
# Esta función de no debe ser modificada.
def PCA_COV(M, m):
    mu = np.mean(M, axis = 0)
    Z = M - mu
    # Computar la matriz de covarianza de Z
    cov_mat = np.dot(Z.T,Z)

    # Computar los valores y vectores propios usando numpy
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    eig_vals = np.real(eig_vals)
    eig_vecs = np.real(eig_vecs)
    
    # Ordenar de forma descendente los vectores propios según sus valores propios asociados
    order = np.argsort(eig_vals)
    order = np.flip(order)
    eig_vals = eig_vals[order]
    eig_vecs = eig_vecs[:,order]
    
    eig_vals = eig_vals[:m]
    V = eig_vecs[:,:m] # V matrix
    Y = M.dot(V)
    return V, Y, mu

# Esta función de no debe ser modificada.
def time_PCA_COV(n, m):
    np.random.seed(0) # Seed para la generación de matrices aleatorias
    M = np.random.random((n,2*n))
    t1 = time.time()
    V, Y, mu = PCA_COV(M,m)
    t2 = time.time()
    tiempo = t2-t1
    return tiempo
    
# Esta función de no debe ser modificada.
def time_PCA_SVD(n, m):
    np.random.seed(0) # Seed para la generación de matrices aleatorias
    M = np.random.random((n,2*n))
    t1 = time.time()
    V, Y, mu = PCA_SVD(M,m)
    t2 = time.time()
    tiempo = t2-t1
    
    return tiempo

In [23]:
def faster(t1, t2):
    is_faster = False if t1 > t2 else True
    #is_faster = np.less(t1, t2, dtype=np.float64)
    return is_faster

In [24]:
def calc_compression_ratio(data_length, M_shape, m1, m2):
    V = M_shape[1]*(m1 + m2)
    Y = M_shape[0]*(m1 + m2)
    mc = V + Y + 2*M_shape[1]

    compression_ratio = (1 - (mc/data_length)) * 100
    return compression_ratio

In [26]:
print(calc_compression_ratio(len(data), M_Re.shape, 18751, 18751))

-29403.135916666666


In [15]:
n = 1000
m = 50

t1 = time_PCA_COV(n,m)
t2 = time_PCA_SVD(n,m)
print(t1, t2)
print(faster(t1, t2))

15.76845121383667 1.037182092666626
False


In [16]:
def compression_algorithm(data, m1, m2):
    M = STFT(data=data)
    M_Re, M_Im = Split(M=M)

    V1, Y1, mu1 = PCA_COV(M_Re, m1)
    V2, Y2, mu2 = PCA_COV(M_Im, m2)
    
    return V1, Y1, mu1, V2, Y2, mu2

In [17]:
t = compression_algorithm(data, 3, 4)
print(t)

(array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
    